In [2]:
import json
import pandas as pd
from pandas import json_normalize


In [3]:
# プレイヤー情報（名前とPUUID）
players = [
    {"name": "민철이여친구함", "puuid": "Tg6htH2UT_o9o11TbmE11RYbJ-o9Uk4iAWU1iaYOu13gc3V0nriz-vqdHfSrRJmhkBC-2ejHAjZ2fA"},
    {"name": "M G", "puuid": "yBObD3pa4qWEzI5v_Uo-azpDPxKgX9c7KbgzeIBw6uQCQmLc3xsZVj8KsEI3JkESDPiYxvKv8MIuDA"},
    {"name": "DK ShowMaker", "puuid": "RaFeXYSCommWUge7Uj_eVGna__co3FOav4nXhVOqjPq2dHGYxa-OlFwT7Om0L-w1c9ShIn9JaOt02Q"},
    {"name": "Yondaime", "puuid": "_DJNQWYxK5yggUNX6lF40G9fibqJZuSOr-JFrE1sWyGON9afLLJKjtqL6TO4Oz0KloQ9rfLi3MB6_g"},
    {"name": "FATE", "puuid": "dXbgFId6t4WGo8XttZ5bcuBq6r-Xycjas9WJNgOMcTR025YA7WM4PO1A3kT-fjYtkyw3v0gFiw8fcg"},
]

In [4]:
# match_data.jsonからデータを取得
with open('match_data.json', 'r', encoding='utf-8') as file:
    matches = json.load(file)

# 結果を格納するリスト
filtered_data = []

# 重複試合を防ぐためのセット
used_match_ids = set()

# 各試合のデータを処理
for match in matches:
    if "info" in match and "participants" in match["info"]:
        match_id = match["metadata"]["matchId"]

        # 同じ試合を2回処理しないためのチェック
        if match_id in used_match_ids:
            continue
        used_match_ids.add(match_id)

        # 試合ごとのプレイヤーデータ
        for participant in match["info"]["participants"]:
            puuid = participant["puuid"]

            # プレイヤーのPUUIDがリストに存在するかチェック
            for player in players:
                if player["puuid"] == puuid:
                    # ゲーム時間（秒）の取得
                    game_length_seconds = participant["challenges"].get("gameLength", 0)

                    # MMSS形式に変換（整数にキャスト）
                    minutes = int(game_length_seconds // 60)
                    seconds = int(game_length_seconds % 60)
                    game_duration = f"{minutes:02d}:{seconds:02d}"

                    # 分間CSの計算
                    cs_per_minute = (participant["totalMinionsKilled"] + participant["neutralMinionsKilled"]) / (game_length_seconds / 60) if game_length_seconds > 0 else 0

                    # Pingsの合計
                    total_pings = sum([
                        participant["allInPings"], participant["assistMePings"], participant["commandPings"],
                        participant["enemyMissingPings"], participant["enemyVisionPings"], participant["holdPings"],
                        participant["needVisionPings"], participant["onMyWayPings"], participant["pushPings"],
                        participant["getBackPings"], participant["visionClearedPings"]
                    ])

                    # 取得するデータに追加
                    filtered_data.append({
                        # 基本情報
                        "試合ID": match["metadata"]["matchId"],
                        "サモナーネーム": player["name"],  # プレイヤー名（リストから取得）
                        "チャンピオン": participant["championName"],
                        "レーン": participant["lane"],
                        "勝敗": "win" if participant["win"] else "lose",
                        "試合時間": game_duration,

                        # パフォーマンス
                        "キル数": participant["kills"],
                        "デス数": participant["deaths"],
                        "アシスト数": participant["assists"],
                        "KDA": participant["challenges"].get("kda", None),
                        "キル関与率": participant["challenges"].get("killParticipation", None),

                        # ダメージ & ゴールド
                        "チャンピオンへの総ダメージ": participant["totalDamageDealtToChampions"],
                        "獲得ゴールド": participant["goldEarned"],
                        "分間ダメージ": participant["challenges"].get("damagePerMinute", None),
                        "分間ゴールド": participant["challenges"].get("goldPerMinute", None),

                        # ファーム & オブジェクト管理
                        "ミニオンキル数": participant["totalMinionsKilled"],
                        "分間CS": cs_per_minute,
                        "ソロキル数": participant["challenges"].get("soloKills", None),

                        # マップコントロール
                        "配置したワード数": participant["wardsPlaced"],
                        "分間ビジョンスコア": participant["challenges"].get("visionScorePerMinute", None),

                        # Pings
                        "総Ping数": total_pings,
                        "オールインPing数": participant["allInPings"],
                        "アシストミーPing数": participant["assistMePings"],
                        "コマンドPing数": participant["commandPings"],
                        "ミアPing数": participant["enemyMissingPings"],
                        "敵の視界Ping数": participant["enemyVisionPings"],
                        "ホールドPing数": participant["holdPings"],
                        "視界必要Ping数": participant["needVisionPings"],
                        "オンマイウェイPing数": participant["onMyWayPings"],
                        "プッシュPing数": participant["pushPings"],
                        "退却Ping数": participant["getBackPings"],
                        "視界クリアPing数": participant["visionClearedPings"],
                    })



In [5]:
# 構造修正
normalized_data = json_normalize(filtered_data)

# プレイヤー名で昇順に並び替え
sorted_df = normalized_data.sort_values(by="サモナーネーム")

# 重複する行を削除（summonerName + gameIdが重複する場合に削除）
sorted_df = sorted_df.drop_duplicates(subset=["サモナーネーム", "試合ID"], keep="first")


In [6]:
#まとめたデータをcsvに出力
sorted_df.to_csv('senseki.csv', index=False, encoding='utf-8-sig')

In [7]:
# プレイヤーごとにデータをグループ化
grouped = sorted_df.groupby('サモナーネーム')

# 各プレイヤーごとにCSVファイルを保存
for player_name, player_data in grouped:
    # プレイヤー名をファイル名に使う
    player_file = f"{player_name}_game_data.csv"
    player_data.to_csv(player_file, index=False, encoding='utf-8-sig')

In [11]:
# 全員のチャンピオンごとのプレイ回数と勝利回数を集計
champion_stats = sorted_df.groupby("チャンピオン").agg(
    プレイ回数=('チャンピオン', 'size'),  # チャンピオンの使用回数
    勝利回数=('勝敗', lambda x: (x == 'win').sum())  # 勝利回数
).reset_index()

# 勝率の計算
champion_stats['勝率'] = champion_stats['勝利回数'] / champion_stats['プレイ回数'] * 100

# CSVに書き出し
champion_stats.to_csv("チャンピオン使用回数と勝率.csv", index=False, encoding="utf-8")



In [13]:
# プレイヤーごとにチャンピオンをグループ化してプレイ回数と勝利回数をカウント
player_champion_stats = sorted_df.groupby(["サモナーネーム", "チャンピオン"]).agg(
    プレイ回数=('チャンピオン', 'size'),  # プレイヤーごとのチャンピオンプレイ回数
    勝利回数=('勝敗', lambda x: (x == 'win').sum())  # 勝利回数
).reset_index()

# 勝率の計算
player_champion_stats['勝率'] = player_champion_stats['勝利回数'] / player_champion_stats['プレイ回数'] * 100

# プレイヤーごとにループして個別CSVを作成
for player_name, player_data in player_champion_stats.groupby("サモナーネーム"):
    # 各プレイヤーのデータをCSVに保存
    player_data.to_csv(f"{player_name}_チャンピオン使用回数と勝率.csv", index=False, encoding="utf-8")
